In [1]:
from requests import get
from xml.etree import ElementTree
import pandas as pd
from datetime import date, datetime
import reverse_geocoder as rg
import time

In [2]:
# XML Path that contains CRE data
xml_prices = 'https://bit.ly/2JNcTha'
xml_geo = 'https://bit.ly/2V1Z3sm'

In [3]:
response_prices = get(xml_prices)
response_geo = get(xml_geo)

In [4]:
prices_tree = ElementTree.fromstring(response_prices.content)
geo_tree = ElementTree.fromstring(response_geo.content)

In [5]:
prices_data = []
geo_data = []

In [6]:
for child in prices_tree:
    for subchild in child:
        dict = {'id_lugar': int(child.attrib['place_id']),
        'Producto': subchild.attrib['type'],
        'Precio': float(subchild.text),
        'Fecha' : date.today().strftime('%Y-%m-%d')
        }
        prices_data.append(dict)

In [7]:
for child in geo_tree:
    dict = {'id_lugar': float(child.attrib['place_id']),
        'Nombre': child.find('name').text,
        'Permiso': child.find('cre_id').text,
        'y' : float(child.find('location').find('y').text), #rg.search(places_obj)[0]['admin1']
        'x' : float(child.find('location').find('x').text)
        }
    geo_data.append(dict)

In [8]:
prices = pd.DataFrame(prices_data)
location = pd.DataFrame(geo_data)

In [9]:
merged_df = pd.merge(how='left',left=prices, right=location, left_on='id_lugar', right_on='id_lugar')
coords = tuple(zip(merged_df['y'], merged_df['x']))

In [10]:
results_rg = rg.search(coords)

Loading formatted geocoded file...


In [11]:
state = [obj.get('admin1') for obj in results_rg]
municipality = []

In [12]:
for obj in results_rg:
    mun = obj.get('admin2')
    if mun == '':
        mun = obj.get('name')
    municipality.append(mun)

In [13]:
merged_df['Estado'] = state
merged_df['Municipio'] = municipality

In [14]:
station_data = pd.read_csv('Resources/Stations_data.csv')

In [15]:
merged_df2 = pd.merge(how='left',left=merged_df, right=station_data, left_on='Permiso', right_on='Numero')
df = merged_df2[['Permiso','Estado','Municipio','EntidadFederativaId','MunicipioId']]

In [23]:
df['EstadoCom'] = df['EntidadFederativaId'].where(df['EntidadFederativaId'].notnull(), df['Estado'])

<ipython-input-23-88c3284642c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EstadoCom'] = df['EntidadFederativaId'].where(df['EntidadFederativaId'].notnull(), df['Estado'])


In [24]:
df[df['EntidadFederativaId'].isnull()]

,Permiso,Estado,Municipio,EntidadFederativaId,MunicipioId,EstadoCom
372,PL/9747/EXP/ES/2015,Jalisco,Valle de Guadalupe,NaN,NaN,Jalisco
373,PL/9747/EXP/ES/2015,Jalisco,Valle de Guadalupe,NaN,NaN,Jalisco
389,PL/9740/EXP/ES/2015,Chihuahua,San Francisco de Conchos,NaN,NaN,Chihuahua
390,PL/9740/EXP/ES/2015,Chihuahua,San Francisco de Conchos,NaN,NaN,Chihuahua
391,PL/9740/EXP/ES/2015,Chihuahua,San Francisco de Conchos,NaN,NaN,Chihuahua
...,...,...,...,...,...,...
32962,PL/10220/EXP/ES/2015,Coahuila,Monclova,NaN,NaN,Coahuila
32963,PL/10220/EXP/ES/2015,Coahuila,Monclova,NaN,NaN,Coahuila
33244,PL/10118/EXP/ES/2015,Durango,Guadalupe Victoria,NaN,NaN,Durango
33284,PL/10102/EXP/ES/2015,Michoacan,Huandacareo,NaN,NaN,Michoacan
